<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 2 - CSP <br>
<font color=696880 size=4>
    Yalda Shabanzadeh, Ali Salesi

____

# Personal Data

In [162]:
# Set your student number
student_number = '98170646'
name = 'Mohammadreza'
last_name = 'Ahmadi Teshnizi'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q1: Cryptarithmetic Puzzle (100 Points)

<font size=4>
Author: Yalda Shabanzadeh, Ali Salesi
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

## Problem Definition

The Cryptarithmetic Puzzle is a puzzle where the digits of some numbers are replaced with letters to make a mathematical equation. Each letter represents a unique digit from 0 to 9 (in case of base 10). The puzzle has the following constraints:

- Each letter can be replaced by a digit from 0 to 9 (in case of base 10), but no digit can be used twice.
- Each letter must be replaced by the same digit throughout the puzzle.
- The leftmost digit of each number cannot be zero.

### Example
Here's an example puzzle:

```
  two
+ two
------
 four
```

In this puzzle, we need to replace each letter with a digit from 0 to 9 (in case of base 10) to make the equation true.

## Note
- You can use any arbitrary CSP algorithm to solve this question.
- Allowed algorithms:
BackTracking, AC3, AC3-LCV, AC3-MRV, AC3-LCV-MRV

> Explain how you consider this problem as a CSP (5 points)


<font color='cyan'>
In this problem, we are given a set of variables: T, W, O, F, U, and R. These variables are considered basic, and each one cannot be assigned the same digit as another variable. The domains for these variables are defined as the set of integers from 0 to 9.
Additionally, there are auxiliary variables X1 and X2, which represent carry values and can take on either 0 or 1. The goal is to find an assignment of values to the variables that satisfies certain arithmetic constraints.
The first constraint: the sum of two O variables should be equal to the sum of R and 10 times X1, taking into account any carry values.
The second constraint: sum of two W variables and the carry value X1 should be equal to the sum of U and 10 times X2.
Finally, the third constraint: sum of two T variables and the carry value X2 should be equal to the sum of O and 10 times F.
To illustrate these constraints and their relationships, a constraint graph can be created. In this graph, each variable is represented as a node, and the constraints are represented as edges connecting the nodes. The goal is to connect all variables in one equation using a common node.
Considering each letter as a variable, we can model the problem using (CSP) techniques. Multiple constraints are assigned to each variable, including a unary constraint that specifies the variable cannot be assigned zero as its first letter. There is also a multiary constraint that ensures the variables in the same column sum up to match the expected result.
</font>

## Input
  - The first line is $N$ ,base of the sum operation. $2 \le N \le 36$. implementing CSP for $N$ other than 10 is optional and test cases with $N$ more than 10 has bonus score since it has a bigger search dimension so AC3, MRV and LCV is suggested.
  - In the following lines until the last, operands for the sum operation is given.
  - In the last line, $result$, the sum of strings is given.
  - **Note**: all the operands and the result are in lowercase.
 
$$1 \le N \le 50$$

### Sample Input
This sample describes the previous example.
```
10
two
two
four
```

## Output
for each given charachter as input, print it's corresponding number:
  - **Note**: order is not important. there are multiple solutions for this problem. if all the constraints are satisfied, you can return any solution.

### Sample Output
The solution to the example above is:
```
f 1
o 4
r 8
t 7
u 6
w 3
```

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. If you tried to use another structure, the total score of that part won't be changed.

### Define Constraint (20 points)

In this part, write the satisfaction part of the constraint of this problem.
You can add any argument you want to `__init__` function.


- Check if all letters have a different value. (Unique Constraint)
- Check if the first letter of each word is not 0. (Not Zero Constraint)
- Check if the sum of the operands with assigned values is equal to the result. (Sum Constraint)
    - **Note**: if there are unassigned values for variables, you can ignore them and return true. for large inputs, you can use AC3 or MRV, LCV to reduce the search space with partial assignment.

In [163]:
import numpy as np
from typing import Dict, List

In [164]:
from typing import Dict, List

class Constraint:
    """
    Base class for all constraints.
    """

    def __init__(self, variable: str) -> None:
        """
        Initializes the constraint.

        Args:
            variable: name of the variable
        """
        self.variable = variable

    def satisfied(self, assignment: Dict[str, int]) -> bool:
        """
        Checks if the constraint is satisfied.

        Args:
            assignment: Dictionary of the assignment of the variables to the values

        Returns:
            False if the constraint is not satisfied, True otherwise (including if the variable is not assigned)
        """
        return True


class NotZeroConstraint(Constraint):
    """"
    Constraint that checks if the variable is not zero.
    """
    def satisfied(self, assignment: Dict[str, int]) -> bool:
        if self.variable not in assignment:
            return True
        return assignment[self.variable] != 0


class UniqueConstraint(Constraint):
    """
    Constraint that checks if all the variables have unique values.
    """
    def __init__(self) -> None:
        pass

    def satisfied(self, assignment: Dict[str, int]) -> bool:
        return len(assignment.keys()) == len(set(assignment.values()))


class SumConstraint(Constraint):
    """
    Constraint that checks if the sum of the operands is equal to the result.
    """

    def __init__(self, operands: List[str], result: str, base: int = 10, allow_carry: bool = False) -> None:
        """
        Initializes the constraint.

        Args:
            operands: List of the operands
            result: result of the sum
            base: base of the numbers
            allow_carry: if true, the variables in the result are checked from right to left so the sum of the operands can have a carry,
                         it is useful for partial sums of columns in AC3 or backtracking. Implementation of this part is not required.
        """
        self.operands = operands
        self.result = result
        self.base = base
        self.allow_carry = allow_carry
        self.word_length = len(operands[0])

    def column_sum(self, assignment: Dict[str, int], column_from_right: int):
        total = 0
        for word in self.operands:
            letter = word[-column_from_right]
            total += assignment[letter]
        return total

    def is_column_assigned(self, assignment: Dict[str, int], column_from_right: int):
        for word in self.operands:
            letter = word[-column_from_right]
            if letter not in assignment:
                return False
        return self.result[-column_from_right] in assignment

    def remaining_result(self, assignment: Dict[str, int]):
        result = 0
        base = 1
        remaining_part = self.result[0:-self.word_length]
        corresponding_value = [assignment[x] for x in remaining_part]
        for i in corresponding_value[::-1]:
            result += i * base
            base *= self.base
        return result

    def satisfied(self, assignment: Dict[str, int]) -> bool:
        carry = 0
        for i in range(1, self.word_length + 1):
            if not self.is_column_assigned(assignment, i):
                return True
            
            result = assignment[self.result[-i]]
            total = self.column_sum(assignment, i)
            
            if (total + carry) % self.base != result:
                return False
            
            carry = (total + carry - result) // self.base
        
        return carry == self.remaining_result(assignment)


### Define a CSP class (50 points)

You can add functions you'll need to this class.

In [165]:
from typing import Dict, List

class CSP:
    """
    CSP class that contains the variables, constraints, and base of the problem.
    """

    def __init__(self, variables: List[str], constraints: List[Constraint], base: int = 10) -> None:
        self.variables = variables
        self.constraints = constraints
        self.base = base
        self.domain = {v: [i for i in range(base)] for v in variables}

    def consistent(self, assignment: Dict[str, int]) -> bool:
        """
        Checks if the assignment is consistent with the constraints.

        Args:
            assignment: Dictionary of the assignment of the variables to the values.

        Returns:
            False if the assignment is not consistent, True otherwise
        """
        for constraint in self.constraints:
            if not constraint.satisfied(assignment):
                return False
        return True

    def choose_unassigned_variable(self, assignment: Dict[str, int]):
        return self.get_unassigned_variables(assignment).pop()

    def get_MRV(self, assignment: Dict[str, int]):
        all_unassigned = self.get_unassigned_variables(assignment)
        all_unassigned.sort(key=lambda x: len(self.domain[x]), reverse=True)
        return all_unassigned.pop()

    def get_unassigned_variables(self, assignment: Dict[str, int]):
        unassigned = set(self.variables) - set(assignment.keys())
        return list(unassigned)

    def update_domain(self, variable, value, reduce=True):
        for v in self.variables:
            if not v == variable:
                if reduce and value in self.domain[v]:
                    self.domain[v].remove(value)
                elif not value in self.domain[v]:
                    self.domain[v].append(value)

    def is_any_domain_empty(self):
        for d in self.domain.values():
            if len(d) == 0:
                return True
        return False

    def backtracking_search(self, assignment: Dict[str, int]):
        """
        Backtracking search algorithm.

        Args:
            assignment: Dictionary of the assignment of the variables to the values. assignment is modified in place.

        Returns:
            True if a solution is found, False otherwise
        """
        if len(assignment.keys()) == len(self.variables):
            return True
        v = self.get_MRV(assignment)
        for i in range(self.base):
            assignment[v] = i
            self.update_domain(v, i, True)
            if not self.is_any_domain_empty():
                if self.consistent(assignment):
                    result = self.backtracking_search(assignment)
                    if result:
                        return result
            self.update_domain(v, i, False)
            assignment.pop(v)
        return False

### Build CSP and solve it! (20 points)

complete these functions for testing.

In [166]:
def get_problem(operands: List[str], result: str, base: int = 10) -> CSP:
    """
    Creates a CSP problem from the operands and the result. The variables are the unique characters in the operands and the result.
    The constraints should be defined using NotZeroConstraint for leftmost digits, a UniqueConstraint for all variables, and a SumConstraint for the problem.
    Multiple partial SumConstraint can be used for bigger search spaces and base > 10.

    Args:
        operands: List of the operands
        result: result of the sum
        base: base of the numbers

    Returns:
        CSP problem
    """
    all_strings = operands + [result]
    variables = list(set("".join(all_strings)))
    leftmost_characters = [w[0] for w in all_strings]
    non_zero_constraints = [NotZeroConstraint(v) for v in set(leftmost_characters)]
    constraints = non_zero_constraints + [UniqueConstraint(), SumConstraint(operands, result, base, True)]
    return CSP(variables, constraints, base)


In [167]:
def backtracking_search(csp: CSP) -> Dict[str, int]:
    """
    Backtracking search algorithm.

    Args:
        csp: CSP problem

    Returns:
        Dictionary of the assignment of the variables to the values if a solution is found, None otherwise
    """
    assignment = {}
    is_solution_found = csp.backtracking_search(assignment)
    if is_solution_found:
        return assignment
    return None


## Test

**Do not change the following cell.**

**Note**: 5 tests will be added after deadline. (5 points).

In [168]:
import helper.test_q2 as q2
import time

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2')
tests_passed = 0

for test in tests:
    base, operands, result = q2.scan_test_input(test)
    csp = get_problem(operands, result, base)
    start_time = time.time()
    sol = backtracking_search(csp)
    if sol is None:
        print(f'test {test} failed. time elapsed={time.time() - start_time:.5f}s')
        continue
    total_time = time.time() - start_time
    if q2.is_result_valid(sol, operands, result, base) and total_time < TIME_LIMIT:
        tests_passed += 1
        print(f'test {test} passed. time elapsed={total_time:.5f}s')
        print('solution: ')
        print('\n'.join([str(x).upper() + " " + str(y)
              for x, y in sorted(sol.items())]))
    else:
        print(f':: test {test} failed. time elapsed={total_time:.5f}s')
print(f'Score = {tests_passed / len(tests) * 100}%')


test q2_in1.txt passed. time elapsed=0.00000s
solution: 
X 1
Y 8
Z 2
test q2_in2.txt passed. time elapsed=0.00803s
solution: 
A 1
B 9
C 8
D 2
E 0
test q2_in3.txt passed. time elapsed=0.00000s
solution: 
A 1
B 9
C 8
test q2_in4.txt passed. time elapsed=0.19200s
solution: 
A 1
F 4
L 0
W 2
Y 5
test q2_in5.txt passed. time elapsed=0.00000s
solution: 
G 8
O 1
T 2
U 0
Score = 100.0%
